In [ ]:
# Import Required Libraries for Object Detection
# Purpose: Set up necessary Python libraries for transfer learning with VGG16

# os: Operating system interface
# - Used for file and path operations

# matplotlib.pyplot: Visualization library
# - For plotting training progress and results

# tensorflow: Deep learning framework
# - Core library for neural networks
# - Provides high-level Keras API

# Key Keras components:
# - layers: Building blocks for neural networks
# - models: Ways to organize layers
# - optimizers: Algorithms to train networks
# - VGG16: Pre-trained CNN for image recognition
# - Flatten: Converts 2D features to 1D
# - Dense: Fully connected layer
# - Dropout: Regularization layer to prevent overfitting

import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout

In [ ]:
# Configuration Settings
# Purpose: Define key parameters for model training

# DATASET_DIR: Path to Caltech-101 dataset
# - Contains images organized in class folders

# VGG_WEIGHTS_PATH: Path to pre-trained VGG16 weights
# - 'notop' indicates weights without classification layers
# - Used for transfer learning

# IMG_SIZE: Input image dimensions (224, 224)
# - Standard size for VGG16
# - All images will be resized to this

# BATCH_SIZE: Number of images processed together
# - 32 is a common choice balancing memory and speed
# - Affects training stability and speed

# SEED: Random seed for reproducibility
# - Ensures consistent train/validation splits
# - Makes results reproducible

DATASET_DIR = "Downloads/LP-IV-datasets-20251107T052642Z-1-001/LP-IV-datasets/Object Detection(Ass6)/caltech-101-img"
VGG_WEIGHTS_PATH = "Downloads/LP-IV-datasets-20251107T052642Z-1-001/LP-IV-datasets/Object Detection(Ass6)/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 1337

In [ ]:
# Data Loading and Preprocessing
# Purpose: Load and prepare image data for training

# Create training dataset:
# Parameters:
# - validation_split=0.2: 20% of data for validation
# - subset="training": Get training portion of split
# - seed=SEED: For reproducible splitting
# - image_size=IMG_SIZE: Resize all images
# - batch_size=BATCH_SIZE: Images per batch
# - label_mode='categorical': One-hot encoded labels
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'   # For categorical_crossentropy loss
)

# Create validation dataset with same parameters
val_data = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# Get class information
class_names = train_data.class_names
num_classes = len(class_names)
print(f"Found {num_classes} classes. Example: {class_names[:8]}")

# Optimize data pipeline:
# cache(): Keeps data in memory after first epoch
# prefetch(): Prepares next batch while current processes
AUTOTUNE = tf.data.AUTOTUNE
train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
val_data = val_data.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Load and Configure VGG16 Base Model
# Purpose: Set up pre-trained VGG16 for transfer learning

# Load VGG16 model:
# Parameters:
# - weights=VGG_WEIGHTS_PATH: Use pre-trained weights
# - include_top=False: Exclude classification layers
# - input_shape=(*IMG_SIZE, 3): Expected image dimensions
base_model = VGG16(weights=VGG_WEIGHTS_PATH, 
                  include_top=False, 
                  input_shape=(*IMG_SIZE, 3))

# Freeze base model layers:
# - Prevents updating VGG16 weights during training
# - Preserves learned feature extractors
base_model.trainable = False

In [ ]:
# Build Custom Classifier
# Purpose: Add classification layers on top of VGG16

model = models.Sequential([
    # Base VGG16 model (frozen)
    base_model,
    
    # Flatten: Convert 3D feature maps to 1D vector
    Flatten(),
    
    # First Dense Layer:
    # - 512 neurons with ReLU activation
    # - Learns high-level features
    Dense(512, activation='relu'),
    
    # First Dropout Layer:
    # - Randomly drops 50% of inputs
    # - Prevents overfitting
    Dropout(0.5),
    
    # Second Dense Layer:
    # - 256 neurons with ReLU activation
    # - Further feature processing
    Dense(256, activation='relu'),
    
    # Second Dropout Layer:
    # - Another 50% dropout for regularization
    Dropout(0.5),
    
    # Output Layer:
    # - num_classes neurons (one per class)
    # - Softmax activation for probability distribution
    Dense(num_classes, activation='softmax')
])

# Display model architecture
model.summary()

In [ ]:
# Model Compilation
# Purpose: Configure training parameters

# Parameters:
# optimizer='adam': 
# - Adaptive learning rate optimization
# - Good default choice for many problems

# loss='categorical_crossentropy':
# - Appropriate for multi-class classification
# - Works with one-hot encoded labels

# metrics=['accuracy']:
# - Track classification accuracy during training
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Initial Model Training
# Purpose: Train the model with frozen VGG16 layers

# Parameters:
# train_data: Training dataset
# validation_data: For monitoring performance
# epochs=8: Number of complete passes through data
# 
# Returns:
# history: Contains training metrics per epoch
# - Useful for plotting learning curves
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=8
)

In [ ]:
# Optional Fine-tuning
# Purpose: Fine-tune last few VGG16 layers for better performance

# Steps (currently commented out):
# 1. Unfreeze last 4 layers of VGG16
# 2. Recompile with very small learning rate (1e-5)
#    - Small rate prevents destroying pre-trained features
# 3. Train for additional epochs
#
# Note: Uncomment to use fine-tuning

# for layer in base_model.layers[-4:]:
#     layer.trainable = True
# model.compile(optimizer=optimizers.Adam(1e-5), 
#              loss='categorical_crossentropy', 
#              metrics=['accuracy'])
# history_ft = model.fit(train_data, 
#                       validation_data=val_data, 
#                       epochs=4)

In [ ]:
# Model Evaluation
# Purpose: Assess model performance on validation set

# model.evaluate returns:
# - loss: Value of loss function
# - acc: Classification accuracy
# 
# Print accuracy as percentage for readability
loss, acc = model.evaluate(val_data)
print(f"Validation Accuracy: {acc*100:.2f}%")

In [ ]:
# Visualize Training Progress
# Purpose: Plot accuracy curves for analysis

# Plot both training and validation accuracy:
# - x-axis: Training epochs
# - y-axis: Accuracy values
# 
# Helps identify:
# - Learning progress
# - Overfitting (if validation accuracy drops)
# - Training stability

plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()